# Optimizing for CO2 storage from CO2 Emitters

## Setup

In [109]:
import numpy as np
import pandas as pd
from pulp import *
from geopy.distance import distance
from geopy.distance import great_circle
import random
import pickle

In [112]:
def facility_cleaning(fac_df, emiss_df):
    facilitiesB = fac_df.rename(inplace=False,
                                    columns={'Facility Name':'facility_name',
                                             'City':'city',
                                             'Primary NAICS Code':'naics_code',
                                             'Industry Type (subparts)':'industry_subparts',
                                             'Industry Type (sectors)':'industry_sectors'})

    emsCA2019loc = pd.merge(emiss_df, facilitiesB[['Facility Id',
                                                    'facility_name',
    #                                               'Facility Name2',
                                                   'city',
                                                   'Zip Code',
                                                   'Address',
                                                   'County',
                                                   'Latitude',
                                                   'Longitude',
    #                                               'Primary NAICS Code2',
                                                   'industry_subparts',
                                                   'industry_sectors']],
                            how='left', on='Facility Id')
    emsCA2019locagg = emsCA2019loc[['Facility Id','facility_name','Latitude','Longitude',
                                    'Unit CO2 emissions (non-biogenic)']].dropna().groupby(
                                        ['Facility Id','facility_name']).agg({'Latitude':'min',
                                                            'Longitude':'min',
                                                            'Unit CO2 emissions (non-biogenic)':'sum'})
    emsCA2019locagg.reset_index(inplace=True)
    emsCA2019locagg = emsCA2019locagg.rename(columns={'Facility Id':'facility_id',
                                                      'facility_name':'facility_name',
                                                      'Latitude':'lat',
                                                      'Longitude':'lon',
                                                      'Unit CO2 emissions (non-biogenic)':'annual_emissions'})
    emsCA2019locagg = emsCA2019locagg[emsCA2019locagg['annual_emissions'] > 0]
    facility_latlon = emsCA2019locagg[['lat', 'lon']].to_records(index=False)
    facility_ids = emsCA2019locagg['facility_id'].tolist()
    facility_names = emsCA2019locagg['facility_name'].tolist()
    supply = emsCA2019locagg['annual_emissions'].tolist()
    return facility_latlon, facility_ids, facility_names, supply


def get_distances(facility_latlon, pools_latlon):
    pools_distance_matrix = []
    for i in range(len(facility_latlon)):
        distances = []
        for j in range(len(pools_latlon)):
            dist = great_circle(facility_latlon[i], pools_latlon[j]).miles
            distances.append(dist)
        pools_distance_matrix.append(distances)
    return pools_distance_matrix


def get_complex_costs(distance_matrix, pool_volumes, emitter_volumes):
    complex_costs = []
    for i in range(len(distance_matrix)):
        new = []
        for j in range(len(distance_matrix[i])):
            # cost = 2 * ($2*miles)*(volume/25) + 2,000,000
            # cost of truck = $2 per mile X2 for round trip
            # number of trucks = volume / 25
            # cost = cost of trunk * number of trucks + 2,000,000
            volume = min(pool_volumes[j], emitter_volumes[i])
            cost = 4 * distance_matrix[i][j] * volume/25 + 2000000
            new.append(cost)
        complex_costs.append(new)
    return complex_costs


def run_optimization(n_receivers, n_suppliers, supply, demand, costs, distances, facility_names, facility_latlon, pools_ids, pools_latlon, industry_str, year):
    print("Running optimization for {} industry and {} year timeframe".format(industry_str, year))
    supply = [i*year for i in supply]
    
    p_prob = LpProblem('Unbalaced_Transportation_Problem', LpMinimize)
    #costs = pools_distance_matrix_complexcost
    routes = [(i, j) for i in range(n_suppliers) for j in range(n_receivers)]

    x = LpVariable.dicts('X', routes, lowBound=0)
    p_prob += lpSum([x[i, j] * costs[i][j] for i in range(n_suppliers) for j in range(n_receivers)])

    for i in range(n_suppliers):
        p_prob += lpSum([x[i, j] for j in range(n_receivers)]) == supply[i]

    for j in range(n_receivers):
        p_prob += lpSum([x[i, j] for i in range(n_suppliers)]) <= demand[j]

    # Solving problem
    p_prob.solve()
    print("Problem successfully solved")
    
    volumes_moved = [i.varValue for i in p_prob.variables()]
    volume_matrix = []
    k=0
    while k < n_suppliers:
        vols = []
        for i in range(n_receivers):
            j = i + k*n_receivers
            vols.append(volumes_moved[j])
        volume_matrix.append(vols)
        k+=1
    print("Volume matrix created")
        
    results = []
    for i in range(n_suppliers):
        f_name = facility_names[i]
        f_loc = facility_latlon[i]
        for j in range(n_receivers):
            vol = volume_matrix[i][j]
            if vol > 0:
                w_id = pools_ids[j]
                w_loc = pools_latlon[j]
                dist = distances[i][j]
                cost = costs[i][j]
                cpt = cost/vol
                r = [f_name, f_loc, w_id, w_loc, dist, cost, vol, cpt, industry_str, year]
                results.append(r)
    results_df = pd.DataFrame(results, columns = ['facility_name', 'facility_location','pool_id','pool_location','distance','cost','volume', 'cost_per_ton', 'industry', 'timeframe'])
    print("Results created")
    print()
    print("--------------------------------------------")
    
    return results_df


## Data import and cleaning

In [115]:
wells = pd.read_csv("data/wells/AllWells_20210915.csv")

# Oil & Gas, Dry Gas (no liquids), Gas and "Liquefied Gas"
wells_oilgas = wells[ (wells['WellType'] == 'OG')  |
                      (wells['WellType'] == 'DG')  |
                      (wells['WellType'] == 'GAS') |
                      (wells['WellType'] == 'LG') ]
wells_latlon = wells_oilgas[['Latitude', 'Longitude']].to_records(index=False)
wells_ids = wells_oilgas['API'].tolist()
pooled_wells = pd.read_csv("data/pool_volumes.csv")
pools_latlon = pooled_wells[['Latitude', 'Longitude']].to_records(index=False)
pools_ids = pooled_wells['pool_id'].tolist()
pools_volumes = pooled_wells['totalco2'].tolist()
n_pools = len(pools_ids)

In [116]:
emissions = pd.read_csv(r'./data/Emissions by Unit and Fuel Type - csv/UNIT_DATA-Table 1.csv')
emsCA2019 = emissions[emissions['Reporting Year'] == 2019]
facilities = pd.read_excel(r'./data/ghgp_data_2019.xlsx',
                          sheet_name='Direct Emitters', skiprows=3)
# One Michegan emitter lists its CA headquarters. Retag it as Michigan
facilities.loc[facilities['Facility Id'] == 1000594, 'State'] = 'MI'

## Prepare data for optimization

In [153]:
facilities = facilities[(facilities.State == 'CA')]
facilities_cement = facilities[facilities['Cement Production'] > 0]
facilities_oil = facilities[facilities['Petroleum Refining'] > 0]
facilities_energy = facilities[facilities['Electricity Generation'] > 0]
facilities_steel = facilities[facilities['Iron and Steel Production'] > 0]

facility_latlon, facility_ids, facility_names, supply = facility_cleaning(facilities, emsCA2019)
n_facilities = len(facility_ids)

facility_c_latlon, facility_c_ids, facility_c_names, supply_c = facility_cleaning(facilities_cement, emsCA2019)
n_facilities_c = len(facility_c_ids)

facility_o_latlon, facility_o_ids, facility_o_names, supply_o = facility_cleaning(facilities_oil, emsCA2019)
n_facilities_o = len(facility_o_ids)

facility_e_latlon, facility_e_ids, facility_e_names, supply_e = facility_cleaning(facilities_energy, emsCA2019)
n_facilities_e = len(facility_e_ids)

facility_s_latlon, facility_s_ids, facility_s_names, supply_s = facility_cleaning(facilities_steel, emsCA2019)
n_facilities_s = len(facility_s_ids)

facility_total_latlon = np.concatenate((facility_c_latlon, facility_o_latlon, facility_e_latlon, facility_s_latlon))
facility_total_ids = np.concatenate((facility_c_ids, facility_o_ids, facility_e_ids, facility_s_ids))
facility_total_names = np.concatenate((facility_c_names, facility_o_names, facility_e_names, facility_s_names))
supply_total = np.concatenate((supply_c, supply_o, supply_e, supply_s))
n_facilities_total = len(facility_total_ids)

In [149]:
facilitiesB = facilities_cement.rename(inplace=False,
                                    columns={'Facility Name':'facility_name',
                                             'City':'city',
                                             'Primary NAICS Code':'naics_code',
                                             'Industry Type (subparts)':'industry_subparts',
                                             'Industry Type (sectors)':'industry_sectors'})
emsCA2019loc = pd.merge(emsCA2019, facilitiesB[['Facility Id',
                                                    'facility_name',
    #                                               'Facility Name2',
                                                   'city',
                                                   'Zip Code',
                                                   'Address',
                                                   'County',
                                                   'Latitude',
                                                   'Longitude',
    #                                               'Primary NAICS Code2',
                                                   'industry_subparts',
                                                   'industry_sectors']],
                            how='left', on='Facility Id')
emsCA2019locagg = emsCA2019loc[['Facility Id','facility_name','Latitude','Longitude',
                                    'Unit CO2 emissions (non-biogenic)']].dropna().groupby(
                                        ['Facility Id','facility_name']).agg({'Latitude':'min',
                                                            'Longitude':'min',
                                                            'Unit CO2 emissions (non-biogenic)':'sum'})
emsCA2019locagg.reset_index(inplace=True)
emsCA2019locagg = emsCA2019locagg.rename(columns={'Facility Id':'facility_id',
                                                      'facility_name':'facility_name',
                                                      'Latitude':'lat',
                                                      'Longitude':'lon',
                                                      'Unit CO2 emissions (non-biogenic)':'annual_emissions'})
emsCA2019locagg = emsCA2019locagg[emsCA2019locagg['annual_emissions'] > 0]
emsCA2019locagg

,facility_id,facility_name,lat,lon,annual_emissions
0,1002308,CEMEX Construction Materials Pacific LLC,34.622200,-117.100100,12834.3
1,1002431,HANSON PERMANENTE CEMENT,37.318100,-122.091000,464.9
2,1004612,LEHIGH SOUTHWEST CEMENT CO.,40.736900,-122.322300,1072.2
3,1005662,Mitsubishi Cement Corp Cushenbury Cement Plant,34.437557,-116.891034,421.7
4,1006642,NATIONAL CEMENT CO OF CALIFORNIA INC,34.819863,-118.748732,2476.8
5,1006842,CalPortland Company Mojave Plant,35.029298,-118.316236,712.0
6,1007927,CalPortland Company Oro Grande Plant,34.604500,-117.338200,270.5


In [150]:
distances_c = get_distances(facility_c_latlon, pools_latlon)
complex_costs_c = get_complex_costs(distances_c, pools_volumes, supply_c)

distances_o = get_distances(facility_o_latlon, pools_latlon)
complex_costs_o = get_complex_costs(distances_o, pools_volumes, supply_o)

distances_e = get_distances(facility_e_latlon, pools_latlon)
complex_costs_e = get_complex_costs(distances_e, pools_volumes, supply_e)

distances_s = get_distances(facility_s_latlon, pools_latlon)
complex_costs_s = get_complex_costs(distances_s, pools_volumes, supply_s)

distances_total = get_distances(facility_total_latlon, pools_latlon)
complex_costs_total = get_complex_costs(distances_total, pools_volumes, supply_total)

In [189]:
industry_list = ['cement', 'oil_refining', 'steel', 'energy', 'total']
year_list = [1, 5, 10, 20, 30]

inputs_list = []
for i in year_list:
    cement = [n_facilities_c, supply_c, complex_costs_c, distances_c, facility_c_names, facility_c_latlon, 'cement', i]
    oil = [n_facilities_o, supply_o, complex_costs_o, distances_o, facility_o_names, facility_o_latlon, 'oil_refining', i]
    energy = [n_facilities_e, supply_e, complex_costs_e, distances_e, facility_e_names, facility_e_latlon, 'energy', i]
    steel = [n_facilities_s, supply_s, complex_costs_s, distances_s, facility_s_names, facility_s_latlon, 'steel', i]
    total = [n_facilities_s, supply_total, complex_costs_total, distances_total, facility_total_names, facility_total_latlon, 'total', i]
    
    inputs_list.append(cement)
    inputs_list.append(oil)
    inputs_list.append(steel)
    inputs_list.append(energy)
    inputs_list.append(total)

    

## Run optimization and write results

In [190]:
detailed_results = pd.DataFrame()

for i in inputs_list:
    results = run_optimization(n_pools,
                               i[0],
                               i[1],
                               pools_volumes,
                               i[2],
                               i[3],
                               i[4],
                               i[5],
                               pools_ids,
                               pools_latlon,
                               i[6],
                               i[7])
    detailed_results = detailed_results.append(results)


Running optimization for cement industry and 1 year timeframe
Problem successfully solved
Volume matrix created
Results created

--------------------------------------------
Running optimization for oil_refining industry and 1 year timeframe
Problem successfully solved
Volume matrix created
Results created

--------------------------------------------
Running optimization for steel industry and 1 year timeframe
Problem successfully solved
Volume matrix created
Results created

--------------------------------------------
Running optimization for energy industry and 1 year timeframe
Problem successfully solved
Volume matrix created
Results created

--------------------------------------------
Running optimization for total industry and 1 year timeframe
Problem successfully solved
Volume matrix created
Results created

--------------------------------------------
Running optimization for cement industry and 5 year timeframe
Problem successfully solved
Volume matrix created
Results create

In [191]:
# re calculate cost as 2 * ($2*miles)*(volume/25) + 2,000,000
detailed_results['cost'] = 4*detailed_results['distance']*(detailed_results['volume']/25) + 2000000
detailed_results['cost_per_ton'] = detailed_results['cost']/detailed_results['volume']
detailed_results

,facility_name,facility_location,pool_id,pool_location,distance,cost,volume,cost_per_ton,industry,timeframe
0,CEMEX Construction Materials Pacific LLC,"[34.6222, -117.1001]",Asphalto_Any Area_Antelope Shale,"[35.282426, -119.56323450000002]",146.754233,2.002706e+06,115.223100,17381.111222,cement,1
1,CEMEX Construction Materials Pacific LLC,"[34.6222, -117.1001]",Bowerbank_Any Area_No Pool Breakdown,"[35.421670000000006, -119.4112865]",141.956006,2.001750e+06,77.034121,25985.234166,cement,1
2,CEMEX Construction Materials Pacific LLC,"[34.6222, -117.1001]",Buena Vista_Buena Vista Hills_Upper (To-Etcheg...,"[35.1898, -119.461314]",139.418375,2.000124e+06,5.577428,358610.543618,cement,1
3,CEMEX Construction Materials Pacific LLC,"[34.6222, -117.1001]",Cache Creek Gas_Any Area_Starkey,"[38.735106815, -121.72364445000002]",382.501960,2.008459e+06,138.224910,14530.372332,cement,1
4,CEMEX Construction Materials Pacific LLC,"[34.6222, -117.1001]",Calders Corner_Any Area_Stevens,"[35.392538, -119.255733]",133.094814,2.000982e+06,46.122788,43383.808295,cement,1
...,...,...,...,...,...,...,...,...,...,...
230,CEMEX Construction Materials Pacific LLC,"[34.6222, -117.1001]",Wilmington_Fault Block V-B_Upper Terminal,"[33.75836563, -118.2056961]",86.918183,2.035693e+06,2566.556600,793.161105,total,30
231,CEMEX Construction Materials Pacific LLC,"[34.6222, -117.1001]",Wilmington_Fault Block VI_Lower Terminal,"[33.75224686, -118.1952744]",86.780092,2.021929e+06,1579.387000,1280.198897,total,30
232,CEMEX Construction Materials Pacific LLC,"[34.6222, -117.1001]",Winchester Lake Gas_Any Area_Mokelumne River,"[38.45892436, -121.553226]",362.369489,2.021130e+06,364.435700,5545.915673,total,30
233,CEMEX Construction Materials Pacific LLC,"[34.6222, -117.1001]",Woodland Gas (ABD)_pool,"[38.695054, -121.807106]",383.623561,2.110612e+06,1802.099700,1171.196280,total,30


In [192]:
summary_results = detailed_results.groupby(by=['industry','timeframe']).agg(
    volume_transferred=('volume', 'sum'),
    total_cost=('cost', 'sum'),
    total_distance=('distance', 'sum')  
)
summary_results['cost_per_ton'] = summary_results['total_cost']/summary_results['volume_transferred']



In [193]:
detailed_results['volume'] = detailed_results['volume']/1000
detailed_results['cost'] = detailed_results['cost']/1000000

In [194]:
summary_results['volume_transferred'] = summary_results['volume_transferred']/1000
summary_results['total_cost'] = summary_results['total_cost']/1000000

In [195]:
detailed_results

,facility_name,facility_location,pool_id,pool_location,distance,cost,volume,cost_per_ton,industry,timeframe
0,CEMEX Construction Materials Pacific LLC,"[34.6222, -117.1001]",Asphalto_Any Area_Antelope Shale,"[35.282426, -119.56323450000002]",146.754233,2.002706,0.115223,17381.111222,cement,1
1,CEMEX Construction Materials Pacific LLC,"[34.6222, -117.1001]",Bowerbank_Any Area_No Pool Breakdown,"[35.421670000000006, -119.4112865]",141.956006,2.001750,0.077034,25985.234166,cement,1
2,CEMEX Construction Materials Pacific LLC,"[34.6222, -117.1001]",Buena Vista_Buena Vista Hills_Upper (To-Etcheg...,"[35.1898, -119.461314]",139.418375,2.000124,0.005577,358610.543618,cement,1
3,CEMEX Construction Materials Pacific LLC,"[34.6222, -117.1001]",Cache Creek Gas_Any Area_Starkey,"[38.735106815, -121.72364445000002]",382.501960,2.008459,0.138225,14530.372332,cement,1
4,CEMEX Construction Materials Pacific LLC,"[34.6222, -117.1001]",Calders Corner_Any Area_Stevens,"[35.392538, -119.255733]",133.094814,2.000982,0.046123,43383.808295,cement,1
...,...,...,...,...,...,...,...,...,...,...
230,CEMEX Construction Materials Pacific LLC,"[34.6222, -117.1001]",Wilmington_Fault Block V-B_Upper Terminal,"[33.75836563, -118.2056961]",86.918183,2.035693,2.566557,793.161105,total,30
231,CEMEX Construction Materials Pacific LLC,"[34.6222, -117.1001]",Wilmington_Fault Block VI_Lower Terminal,"[33.75224686, -118.1952744]",86.780092,2.021929,1.579387,1280.198897,total,30
232,CEMEX Construction Materials Pacific LLC,"[34.6222, -117.1001]",Winchester Lake Gas_Any Area_Mokelumne River,"[38.45892436, -121.553226]",362.369489,2.021130,0.364436,5545.915673,total,30
233,CEMEX Construction Materials Pacific LLC,"[34.6222, -117.1001]",Woodland Gas (ABD)_pool,"[38.695054, -121.807106]",383.623561,2.110612,1.802100,1171.196280,total,30


In [196]:
list_results = detailed_results.values.tolist()
file_name = 'data/results/full_detailed_results.pkl'
open_file = open(file_name, "wb")
pickle.dump(list_results, open_file)
open_file.close()

html_results = detailed_results.to_html()
html_file = open("data/results/full_detailed_results.html", "w")
html_file.write(html_results)
html_file.close()

In [197]:
total_pool_volume = sum(pools_volumes)/1000

summary_results['percent_pool_volume_used'] = summary_results['volume_transferred'] / total_pool_volume
summary_results

volume_transferred    total_cost  total_distance  \
industry     timeframe                                                     
cement       1                   18.252400    178.572037    18411.660806   
             5                   91.262000    280.854147    26933.056665   
             10                 182.523999    347.925493    33726.292742   
             20                 365.048000    439.904247    43055.013564   
             30                 547.572000    503.348742    48317.452634   
energy       1                27951.615590   2473.871587   158966.726577   
             5               139758.078018   6502.553863   199146.946142   
             10              279516.156666  10950.171655   216748.092135   
             20              559032.312164  19757.777617   225899.307623   
             30              838548.468527  27603.907002   229121.329210   
oil_refining 1                17356.620285   1660.206212   102156.370655   
             5                86783.101484   3698.960662   142639.733831   
             10              173566.203295   6069.673536   152034.781823   
             20              347132.406188   9750.189616   166060.993240   
             30              520698.609329  13472.171185   168681.643963   
steel        1                   33.308600    201.119871    19639.030810   
             5                  166.543000    341.040070    31713.736410   
             10                 333.086000    421.153888    38513.083543   
             20                 666.172000    515.259411    46611.254652   
             30                 999.257997    580.685910    51521.407669   
total        1                   12.834300    156.419269    15867.546960   
             5                   64.171500    267.918546    25811.047685   
             10                 128.342999    332.053881    32124.964092   
             20                 256.686000    420.140982    41172.506568   
             30                 385.028999    481.589166    46434.945638   

                        cost_per_ton  percent_pool_volume_used  
industry     timeframe                                          
cement       1           9783.482528                  0.000010  
             5           3077.448959                  0.000052  
             10          1906.190384                  0.000105  
             20          1205.058642                  0.000210  
             30           919.237548                  0.000315  
energy       1             88.505495                  0.016070  
             5             46.527213                  0.080348  
             10            39.175452                  0.160697  
             20            35.342819                  0.321393  
             30            32.918678                  0.482090  
oil_refining 1             95.652620                  0.009978  
             5             42.623052                  0.049892  
             10            34.970365                  0.099785  
             20            28.087812                  0.199570  
             30            25.873261                  0.299355  
steel        1           6038.076380                  0.000019  
             5           2047.759863                  0.000096  
             10          1264.399850                  0.000191  
             20           773.463026                  0.000383  
             30           581.117101                  0.000574  
total        1          12187.596396                  0.000007  
             5           4175.039494                  0.000037  
             10          2587.237965                  0.000074  
             20          1636.789629                  0.000148  
             30          1250.786737                  0.000221

In [198]:
list_summary_results = summary_results.values.tolist()
file_name = 'data/results/summary_results.pkl'
open_file = open(file_name, "wb")
pickle.dump(list_summary_results, open_file)
open_file.close()

html_summary_results = detailed_results.to_html()
html_file = open("data/results/summary_results.html", "w")
html_file.write(html_summary_results)
html_file.close()